In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# load data

In [ ]:
df= pd.read_csv('supermarket_data.csv') 
df

In [ ]:
df.describe()

Checking the DataSet for null values:

In [ ]:
df.isna().sum()

In [ ]:
df['SHOP_DATE'] = pd.to_datetime(df['SHOP_DATE'], format='%Y%m%d')

# EDA

### How many baskets/transaction are in the data set?

In [ ]:
print('there are : ',len(df['BASKET_ID'].unique()),'baskets/transactions')

### How many products are in the data set?

In [ ]:
print('there are : ',len(df['PROD_CODE'].unique()),'unique products')

In [ ]:
prod_sold = df.groupby(['PROD_CODE'])['QUANTITY'].count().reset_index().rename(columns={'QUANTITY':'PROD_COUNT'})
prod_sold

In [ ]:
total_count_of_items = len(df['PROD_CODE'])
print("Total count of items: ", total_count_of_items)

In [ ]:
sum(df['QUANTITY'])

### Contribution of top "sold products" to total sales

In [ ]:
prod_sold['PROD_PERC'] = prod_sold['PROD_COUNT']/total_count_of_items #each item's contribution 
prod_sold['TOTAL_PERC'] =  prod_sold.PROD_PERC.cumsum()

top_prod_sold = prod_sold.sort_values('PROD_PERC',ascending=False).head(50)
top_prod_sold['TOTAL_PERC'] =  top_prod_sold.PROD_PERC.cumsum() #cumulative contribution of top items

print(prod_sold[prod_sold['TOTAL_PERC'] <= 0.2].shape)

top_prod_sold

This shows us that the top 20 products are responsible for 14.4% of the entire sales and the top 1000 products are responsible for 20% of the sales!

### Monthly revenue

In [ ]:
df['YEAR_MONTH'] = df['SHOP_DATE'].map(lambda date: date.year*100 + date.month)
df_revenue = df.groupby(['YEAR_MONTH']).agg({'SPEND' : 'sum'}).reset_index()
df_revenue.head()

In [ ]:
plot_data = [go.Scatter(
                 x = df_revenue['YEAR_MONTH'],
                 y = df_revenue['SPEND'],
             )]

plot_layout = go.Layout(
    xaxis = {'type': 'category'},
    title = "Monthly Revenue")

fig1 = go.Figure(data=plot_data, layout=plot_layout)
fig1

In [ ]:
df_revenue['MONTHLYGROWTH'] = df_revenue['SPEND'].pct_change()
df_revenue.head()

### Monthly Revenue Growth Rate

In [ ]:
plot_data = [go.Bar(
                 x = df_revenue['YEAR_MONTH'],
                 y = df_revenue['MONTHLYGROWTH'],
             )]

plot_layout = go.Layout(
    xaxis = {'type': 'category'},
    title = "Monthly Revenue Growth Rate")

fig2 = go.Figure(data=plot_data, layout=plot_layout)
fig2

### Monthly Customer

In [ ]:
df_cust = df.groupby('YEAR_MONTH')['CUST_CODE'].nunique().reset_index()
df_cust.head()

In [ ]:
plot_data = [go.Scatter(
                 x = df_cust['YEAR_MONTH'],
                 y = df_cust['CUST_CODE'],
             )]

plot_layout = go.Layout(
    xaxis = {'type': 'category'},
    title = "Monthly Customer")

fig3 = go.Figure(data=plot_data, layout=plot_layout)
fig3

### Monthly Customer Growth Rate

In [ ]:
df_cust['MONTHLYGROWTH'] = df_cust['CUST_CODE'].pct_change()
df_cust.head()

In [ ]:
plot_data = [go.Bar(
                 x = df_cust['YEAR_MONTH'],
                 y = df_cust['MONTHLYGROWTH'],
             )]

plot_layout = go.Layout(
    xaxis = {'type': 'category'},
    title = "Monthly Customer Growth Rate")

fig4 = go.Figure(data=plot_data, layout=plot_layout)
fig4

### Revenue by store

In [ ]:
df_store = df.groupby(['STORE_CODE']).agg({'SPEND' : 'sum'}).reset_index()
df_store

In [ ]:
plot_data = [go.Pie(
                 labels = df_store['STORE_CODE'], 
                 values = df_store['SPEND'],
             )]

plot_layout = go.Layout(
    title = "Revenue by store")

fig5 = go.Figure(data=plot_data, layout=plot_layout)
fig5

# Market base analysis 

Association rules can be applied in many ways, one of the most popular one appears in a shopping baskets data sets, as it can bring valuable information to the shops, and it can answer many questions like: How the catalog of products should be designed? Where each product should be located in the shop? Which products should receive a promotion? Additionally, we can predict customer behavior.

## Preprocessing data

consolidate the items into 1 transaction per row with each product

In [ ]:
basket = df.groupby(['BASKET_ID', 'PROD_CODE'])['QUANTITY']\
        .sum().unstack().reset_index().fillna(0)\
        .set_index('BASKET_ID')
basket

There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less than 0 is set to 0

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [ ]:
basket_sets = basket.applymap(encode_units)
basket_sets

## Generate frequent itemsets


- minimum support = 0.01

 means the itemset occurs in 1% of the transactions.  min_support should be selected as per the requirement of the business.

- minimum confidence = 0.3 or 30%

 means that, whenever the customer bought items in the antecedent, 30% of the times he/she also bought items in the consequents. The higher the min_confidence, the lesser the rules we generate.

- lift > 1

 means that, there is a dependence between the antecedent and consequent



In [ ]:
def determine_min_support(df, expected_txn_per_day):
    min_dt = pd.to_datetime(df['SHOP_DATE'].min(), format='%Y%m%d', errors='coerce')
    max_dt = pd.to_datetime(df['SHOP_DATE'].max(), format='%Y%m%d', errors='coerce')
    delta = max_dt - min_dt

    actual_txn = len(df)
    expected_txn = expected_txn_per_day * delta.days
    
    return expected_txn / actual_txn

min_sup = determine_min_support(df,12)
min_sup = round(min_sup,2)
min_sup

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=min_sup, use_colnames=True)
frequent_itemsets

## Generate rules

In [ ]:
#with minimum support: 0.01 and minimum lift: 1, there are 28 rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print('there are ',len(rules),'rules')

In [ ]:
rules.head()

In [ ]:
#filter rules with minimum confidence = 0.3 and lift > 1 
rules[ (rules['lift'] >= 1) &
      (rules['confidence'] >= 0.3) ]


As we can see in the results above, we obtained a large lift value. As the value in this top is greater than 1, it means that the occurrence of the first product (for example, PRD0900173) has a positive effect on the occurrence of the product (PRD0903052)

Confident of the rule is 38.4%, which means that 38.4% of the time the antecedent items occurred, we also had the consequent in the transaction (i.e., 38.4% of times, customers who bought PRD0900173 also bought PRD0903052).

## Conclusion 

Thus, using the dataset provided, we could generate 5 association rules. This number can be varied by tweaking the parameters like support and confidence. Higher the values, lesser the number of rules and vice versa. 

However, when we set these parameters high, the quality of the generated rules is also high. We have to strike a middle ground between the quality of rules and their quantity.

Ultimately, it all depends on the requirements of the business.

In [ ]:
# #output the results to a csv file
# rules.to_csv("retail_basket.csv")